In [1]:
import pandas as pd
import json
import time
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError
from geopy.geocoders import Nominatim

rentdata_df = pd.read_json('rent_data.json')

In [2]:
title = []                  # 1
price = []                  # 2
area = []                   # 3
bedroom = []        # 4
description = []            # 5
width = []                  # 6
direction = []              # 7
toilet = []         # 8
address = []               # 9
geo_code = []               # 10
law = []
floor = []
district = []
province_city = []
type = []
type_estate = []
image_links = []
phonenumber = []

In [3]:
import requests
import pandas as pd
import re

rentdata_df["geo_code"] = [None for _ in range(len(rentdata_df))]

def get_geocode(address):
    api_key = 'b48b7a0d92434d7ba9e13d0feb2db0ad'
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"

    response = requests.get(url)
    data = response.json()

    if data['results']:
        latitude = data['results'][0]['geometry']['lat']
        longitude = data['results'][0]['geometry']['lng']
        print(f"Latitude: {latitude}, Longitude: {longitude}")
        return [latitude, longitude]
    else:
        # Check if address contains "Đường"
        if "Đường" in address:
            # Remove "Đường" and retry
            modified_address = address.replace("Đường", "").strip()
            print(f"Retry with modified address: {modified_address}")
            return get_geocode(modified_address)
        else:
            print("No results found.")
            return None

def data_normalization(df):
    for i, address in enumerate(df["address"]):
        df.at[i, "geo_code"] = get_geocode(address)
    
    for i, district in enumerate(df["district"]):
        if isinstance(district, str):
            df.at[i, "district"] = district.replace('Huyện ', '').replace('Quận ', '').replace('Thành phố ', '').strip()
        
    for i, price in enumerate(df['price']):
        if isinstance(price, str):
            df.at[i, 'price'] = price.replace(' triệu / tháng', '').replace(',', '.').replace(' ngàn / m2  / tháng', '').strip() 
    
    for i, area in enumerate(df['area']):
        if isinstance(area, str):
            df.at[i, 'area'] = area.replace(' m2', '').strip()
    
    for i, width in enumerate(df['width']):        
        if isinstance(width, str):
            df.at[i, 'width'] = width.replace('m', '').strip()
            
    for i, phonenumber in enumerate(df['phonenumber']):        
        if isinstance(phonenumber, str):
            phonenumber = phonenumber.replace(' ', '').replace('.', '').strip()
            # Kiểm tra nếu số đầu tiên khác 0 thì thêm số 0 vào đầu
            if phonenumber[0] != '0':
                phonenumber = '0' + phonenumber
            df.at[i, 'phonenumber'] = phonenumber.strip()
            
    for i, title in enumerate(df['title']):
        if isinstance(title, str):
            title = re.sub(r'[^\w\s,.()\–]', '', title)
            title = title.lower()
            title = title.capitalize()
            df.at[i, 'title'] = title.strip()    

data_normalization(rentdata_df)

rentdata_df['type_estate'] = rentdata_df['type_estate'].replace({
    'Nhà mặt tiền': 'Nhà Mặt Phố',
    'Biệt thự, nhà liền kề': 'Nhà Biệt Thự, Liền Kề',
    'Nhà trong hẻm': 'Nhà Trong Ngõ',
    'Căn hộ chung cư': 'Căn Hộ Chung Cư',
    'Đất thổ cư, đất ở': 'Nhà Đất Thổ Cư',
    'Đất nền, liền kề, đất dự án': 'Đất Nền Dự Án',
    'Đất nông, lâm nghiệp': 'Đất Nông Nghiệp, Đất Vườn',
    'Trang trại': 'Đất Trang Trại',
    'Văn phòng': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Nhà hàng, khách sạn': 'Nhà Hàng, Khách Sạn, Resort',
    'Shop, kiot, quán': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'Kho, xưởng': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'Mặt bằng': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'Phòng trọ, nhà trọ': 'Nhà Trọ, Phòng Trọ',
})

def write_data(df, field, data_list):
    if not isinstance(data_list, list):
        data_list = []  # Initialize as empty list if not already
    
    if field in df.columns:
        for value in df[field]:
            data_list.append(value)
    else:
        print(f"Error: '{field}' column not found in the DataFrame.")
 
# Populate lists with data from DataFrame
write_data(rentdata_df, "title", title)
write_data(rentdata_df, "price", price)
write_data(rentdata_df, "area", area)
write_data(rentdata_df, "type", type)
write_data(rentdata_df, "type_estate", type_estate)
write_data(rentdata_df, "image_links", image_links)
write_data(rentdata_df, "bedroom", bedroom)
write_data(rentdata_df, "description", description)
write_data(rentdata_df, "width", width)
write_data(rentdata_df, "direction", direction)
write_data(rentdata_df, "toilet", toilet)
write_data(rentdata_df, "address", address)
write_data(rentdata_df, "law", law)
write_data(rentdata_df, "floor", floor)
write_data(rentdata_df, "geo_code", geo_code)
write_data(rentdata_df, "district", district)
write_data(rentdata_df, "province_city", province_city)
write_data(rentdata_df, "phonenumber", phonenumber)

# Display the updated DataFrame
print("\nUpdated DataFrame:")


Latitude: 10.8149344, Longitude: 106.7080486
Latitude: 10.0145361, Longitude: 105.7563479
Latitude: 21.0020266, Longitude: 105.8190073
Latitude: 21.0461332, Longitude: 105.8019628
Latitude: 16.027134, Longitude: 108.2438914
Latitude: 21.0388103, Longitude: 105.7818897
Latitude: 10.8412537, Longitude: 106.7258101
Latitude: 20.92925, Longitude: 105.83584
Latitude: 21.0070778, Longitude: 105.7734833
Here is the extracted information in the format you requested:

`law`: Sổ pháp lý
`width`: 150 (mặt tiền đường lớn 2 chiều)

Let me know if you need any further assistance!
Error parsing response: Expecting value: line 1 column 1 (char 0)
Here is the extracted information in the format you requested:

{
"law": "",
"width": 10.2m
}

Note that there is no specific information about the "law" (sổ pháp lý) in the provided text, so it's left blank. The width (mặt tiền) is 10.2m.
Error parsing response: Expecting value: line 1 column 1 (char 0)
Here is the information about the office for rent in th

In [4]:
rentdata_df

,title,description,price,area,address,province_city,district,direction,type,bedroom,toilet,floor,law,type_estate,width,image_links,geo_code
0,"TOÀ MỚI, HẦM VÀ SÂN RỘNG PHÙ HỢP VĂN PHÒNG, TT...",chủ tòa nhà văn phòng QCOOP 150 Nguyễn Xí (mặt...,340,400,"150 Đường Nguyễn Xí, Phường 26, Quận Bình Thạ...",Hồ Chí Minh,Bình Thạnh,Tây Nam,Cho Thuê,NaN,NaN,NaN,None,"Văn phòng, TTTM, Cửa hàng, Kiot",None,https://alonhadat.com.vn//files/properties/202...,"[10.8149344, 106.7080486]"
1,💥💥CHO THUÊ MẶT BẰNG KINH DOANH MẶT TIỀN ĐƯỜNG ...,💥💥CHO THUÊ MẶT BẰNG KINH DOANH MẶT TIỀN ĐƯỜNG ...,80,800,"Đường 3 tháng 2, Phường Hưng Lợi, Quận Ninh K...",Cần Thơ,Ninh Kiều,None,Cho Thuê,NaN,NaN,NaN,Sổ hồng/ Sổ đỏ,"Nhà Xưởng, Mặt Bằng, Kho Bãi","10,2",https://alonhadat.com.vn//files/properties/202...,"[10.0145361, 105.7563479]"
2,chủ cho thuê văn phòng tại tòa nhà 8 tầng 51 N...,cho thuê văn phòng tại tòa nhà 8 tầng 51 Nguyễ...,5,35,"51 Đường Nguyễn Trãi, Phường Khương Trung, Qu...",Hà Nội,Thanh Xuân,Tây Bắc,Cho Thuê,NaN,NaN,8.0,Sổ hồng/ Sổ đỏ,"Văn phòng, TTTM, Cửa hàng, Kiot",5,https://alonhadat.com.vn//files/properties/202...,"[21.0020266, 105.8190073]"
3,GIẢM GIÁ 30% - THUÊ VĂN PHÒNG 30M2 - 50M2 - 10...,- Vị trí đẹp : Gần ngã tư và nhiều ngân hàng t...,6.9,30,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận ...",Hà Nội,Cầu Giấy,None,Cho Thuê,NaN,NaN,30.0,None,"Văn phòng, TTTM, Cửa hàng, Kiot",25,https://alonhadat.com.vn//files/properties/202...,"[21.0461332, 105.8019628]"
4,Cho thuê nhà sân vườn đối diện công viên đường...,Cho thuê nhà sân vườn đối diện công viên đường...,8,100,"Đường Mạc Cửu, Phường Khuê Mỹ, Quận Ngũ Hành ...",Đà Nẵng,Ngũ Hành Sơn,Tây Bắc,Cho Thuê,1.0,1.0,1.0,Giấy tờ hợp lệ,Nhà Mặt Phố,5,https://alonhadat.com.vn//files/properties/202...,"[16.027134, 108.2438914]"
5,"Cho thuê phòng khu Dịch vọng hậu, Cầu Giấy, fu...","Cho thuê phòng full đồ, khép kín, nội thất cao...",4,25,"Đường Xuân Thủy, Phường Dịch Vọng Hậu, Quận C...",Hà Nội,Cầu Giấy,None,Cho Thuê,1.0,1.0,1.0,Sổ hồng/ Sổ đỏ,"Nhà Trọ, Phòng Trọ",5,https://alonhadat.com.vn//files/properties/202...,"[21.0388103, 105.7818897]"
6,"Cho Thuê Văn Phòng, Diện Tích: 10x20=200m2 Giá...","CHO THUÊ NHÀ khu Bình Dân, Hiệp Bình Chánh, Th...",22,200,"Đường B, Phường Hiệp Bình Chánh, Thành phố Th...",Hồ Chí Minh,Thủ Đức,Bắc,Cho Thuê,4.0,4.0,1.0,Sổ hồng/ Sổ đỏ,"Văn phòng, TTTM, Cửa hàng, Kiot",10,https://alonhadat.com.vn//files/properties/202...,"[10.8412537, 106.7258101]"
7,"Cho thuê 400m thanh trì, xe container chạy vòn...","cho thuê 400m2 tại đường Miếu Nghè, Thanh Trì,...",70,400,"Đường Phan Trọng Tuệ, Xã Vĩnh Quỳnh, Huyện Th...",Hà Nội,Thanh Trì,None,Cho Thuê,NaN,NaN,NaN,None,"Nhà Xưởng, Mặt Bằng, Kho Bãi",None,https://alonhadat.com.vn//files/properties/202...,"[20.92925, 105.83584]"
8,"Cho thuê 230m2 kho Mỹ Đình, điện 3fa, xe 3,5t ...","Cho thuê 230m2 kho xưởng, mặt tiền rộng, tại M...",20,230,"Đường Lê Quang Đạo, Phường Phú Đô, Quận Nam T...",Hà Nội,Nam Từ Liêm,None,Cho Thuê,NaN,NaN,NaN,None,"Nhà Xưởng, Mặt Bằng, Kho Bãi",None,https://alonhadat.com.vn//files/properties/202...,"[21.0070778, 105.7734833]"


In [5]:
rentdata_df.to_csv('rent_data.csv', encoding='utf-8', index=False)

In [6]:
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with connection.cursor() as cursor:
        # Xóa dữ liệu từ cả hai bảng pools_bds và pools_image
        cursor.execute("""
            DELETE pb, pi
            FROM pools_bds pb
            JOIN pools_image pi ON pb.BDS_id = pi.BDS_id
            WHERE pb.kind = 'Cho Thuê' AND pi.content LIKE '%nhadat.com%'
        """)
        connection.commit()

finally:
    connection.close()


In [8]:
import csv
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL với mã hóa utf8mb4
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

def is_decimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

try:
    with connection.cursor() as cursor:
        
        # Đọc dữ liệu từ file CSV và nhập vào MySQL
        with open('rent_data.csv', 'r', encoding='utf-8') as file:
            csv_data = csv.DictReader(file)
            for row in csv_data:
                try:
                    district = row['district']
                    province_city = row['province_city']
                    
                    # Kiểm tra tồn tại address
                    cursor.execute("SELECT address_id FROM pools_address WHERE district = %s AND province_city = %s", (district, province_city))
                    existing_address = cursor.fetchone()
                    if existing_address:
                        address_id = existing_address['address_id']
                    else:
                        # Nếu không tìm thấy địa chỉ, bỏ qua dòng này
                        print(f"Không tìm thấy địa chỉ cho dòng: {row}")
                        continue
                    
                    # Kiểm tra pools_typeofbds 
                    bds_type = row['type_estate']
                    cursor.execute("SELECT Type_Id FROM pools_typeofbds WHERE TypeName = %s", (bds_type,))
                    existing_type = cursor.fetchone()
                    if existing_type:
                        type_id = existing_type['Type_Id']
                    else:
                        # Nếu không tìm thấy loại bất động sản, bỏ qua dòng này
                        print(f"Không tìm thấy loại bất động sản cho dòng: {row}")
                        continue
                    
                    # Thêm dữ liệu vào bảng pools_bds
                    title = row['title']
                    price = row['price']
                    area = row['area']
                    kind = row['type']
                    law = row['law']
                    bedroom = row['bedroom']
                    floor = row['floor']
                    description = row['description']
                    width = row['width']
                    direction = row['direction']
                    toilet = row['toilet']
                    location = row['address']
                    geocode = row['geo_code']
                    
                    # Handle 'price' column
                    if price.strip().lower() == 'thỏa thuận':
                        price = 0  # or set to a default value, like 0
                    elif not is_decimal(price):
                        price = 0  # or set to a default value

                    # Handle 'width' column
                    if not is_decimal(width):
                        width = 0  # or set to a default value

                    # Handle 'bedroom' column
                    if not bedroom.strip():
                        bedroom = 0  # or set to a default value
                       
                    # Handle 'floor' column 
                    if not floor.strip():
                        floor = 0  # or set to a default value

                    # Handle 'toilet' column (if needed)
                    if not is_decimal(toilet):
                        toilet = 0  # or set to a default value

                    # Kiểm tra geocode nếu là None hoặc rỗng, bỏ qua dòng này
                    if not geocode:
                        # print(f"Không có giá trị geocode cho dòng: {row}")
                        continue

                    # Chuyển geo_code thành string nếu nó là list
                    if isinstance(geocode, list):
                        geocode = str(geocode)

                    # Thêm dữ liệu vào bảng pools_bds
                    cursor.execute("""
                        INSERT INTO pools_bds (title, price, area, kind, law, bedroom, floor, address_id, Type_id, description, width, direction, toilet, location, geocode)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (title, price, area, kind, law, bedroom, floor, address_id, type_id, description, width, direction, toilet, location, geocode))
                    connection.commit()
                    
                    # Lấy BDS_id của BDS vừa thêm
                    cursor.execute("SELECT LAST_INSERT_ID()")
                    bds_id = cursor.fetchone()['LAST_INSERT_ID()']
                    
                    # Thêm dữ liệu vào bảng pools_image (nếu có)
                    if 'image_links' in row and row['image_links']:  # Kiểm tra xem cột 'image_link' có tồn tại và có giá trị không
                        image_link = row['image_links']
                        cursor.execute("INSERT INTO pools_image (BDS_id, content) VALUES (%s, %s)", (bds_id, image_link))
                        connection.commit()
                        
                except Exception as e:
                    print(f"Lỗi khi xử lý dòng: {row}")
                    print(e)
finally:
    connection.close()
